## Temporal

**Observations**  
-Remplissage  


In [ ]:
import pandas as pd

from utils import wrapper_engine, get_value_from_json_serialization, date_string_to_datetime
engine = wrapper_engine('config.ini')

import custom_settings
from itables import show
custom_settings.apply_itable_custom_settings()

In [ ]:
# Load data
with engine.connect() as connection:
    package = pd.read_sql_table(
        table_name="package", 
        con=connection,
        schema="public",
        )
    
    package_extra = pd.read_sql_table(
        table_name="package_extra", 
        con=connection,
        schema="public",
        )
    
temporal = package_extra[package_extra["key"]=="temporal"].copy(deep=True)

**Nombre de datasets avec la métadonnée 'temporal'**

In [ ]:
show(temporal[["package_id", "value"]].rename(columns={"package_id":"datasets", "value":"temporal"}).describe())

**Datasets avec la métadonnée temporal**

In [ ]:
datasets_with_temporal = temporal.merge(package, how="inner", left_on="package_id", right_on="id")
show(datasets_with_temporal[["title", "notes", "value"]].rename(columns={"value":"temporal"}))

In [ ]:
datasets_with_temporal["start_date"] = datasets_with_temporal["value"].apply(lambda x: get_value_from_json_serialization(x, key="start_date")[0]).apply(lambda x: date_string_to_datetime(x))
datasets_with_temporal["end_date"] = datasets_with_temporal["value"].apply(lambda x: get_value_from_json_serialization(x, key="end_date")[0]).apply(lambda x: date_string_to_datetime(x))

In [ ]:
datasets_with_temporal.sort_values(by=["start_date", "end_date"], inplace=True)
datasets_with_temporal["Datasets"] = [i for i in range(len(datasets_with_temporal))]

import plotly
import plotly.express as px
plotly.offline.init_notebook_mode(connected=True)

fig = px.timeline(datasets_with_temporal, 
                  x_start="start_date", 
                  x_end="end_date", 
                  y="Datasets", 
                  hover_data=["title"],
                  color_discrete_sequence=['#000091'])
fig.update_layout(bargap=0.2,
                  yaxis_title="Datasets",
                  xaxis_title="temporal")
fig.update_traces(opacity=0.8)
fig.update_yaxes(autorange="reversed")
plotly.offline.iplot(fig)